# 1. Importing some Libraries

In [ ]:
import numpy as np
import pandas as pd
from influxdb import InfluxDBClient
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


from tensorflow.keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from sklearn import metrics
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


# 2. Fetching Data: DE1Thing_HMD8310

In [ ]:
# Connect to InfluxDB and fetch data
client = InfluxDBClient(host='localhost', port=8086)
client.switch_database('ISS')

# Query to the Database for one measurement

query1 = 'SELECT * FROM "DE1Thing_HMD8310"'
#query1 = 'SELECT * FROM "DE1Thing_HMD8310" WHERE time >= \'2022-08-29T23:28:00Z\' AND time < \'2023-01-26T08:00:00Z\''
#query1 = 'SELECT * FROM "DE1Thing_HMD8310" WHERE time >= \'2022-09-30T00:00:00Z\' AND time < \'2022-10-30T23:59:00Z\''
results1 = client.query(query1)
DE1Thing_HMD8310 = pd.DataFrame.from_records(results1.get_points())

print(DE1Thing_HMD8310.shape)
DE1Thing_HMD8310.head(2)

In [ ]:
DE1Thing_HMD8310.to_csv("Data/DE1Thing_HMD8310.csv")

In [ ]:
# Function to convert ISO 8601 time to datetime
def ISO_8601_To_Datetime(s):
    return datetime.strptime(s, '%Y-%m-%dT%H:%M:%SZ')

In [ ]:
# Read The data
df = pd.read_csv('Data/DE1Thing_HMD8310.csv', parse_dates=[1], index_col=0, date_parser=ISO_8601_To_Datetime)
df.head(2)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Select only 21 features
df = df.loc[:, ['time',
                'CFWInletPress',
                'CFWInletTemp', 
                'CFWOutletTempA', 
                'CFWOutletTempB',
                'Cy1ExhGasOutletTemp',
                'Cy2ExhGasOutletTemp',
                'Cy3ExhGasOutletTemp',
                'Cy4ExhGasOutletTemp',
                'Cy5ExhGasOutletTemp',
                'Cy6ExhGasOutletTemp',
                'Cy7ExhGasOutletTemp',
                'Cy8ExhGasOutletTemp',
                'Cy9ExhGasOutletTemp',
                'FGTemperature',
                'FOInletPress',
                'FOInletTemp',
                'InletAirTemp',
                'LOInletTemp',
                'ScavPress',
                'ScavTemp']]
df.head(2)

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df

In [ ]:
# Checking for blank values and Data Types.
def overview(df: pd.DataFrame, timestamp_col: str= None) -> None:
    print('Null Count:\n', df.isnull().sum(), '\n')
    print('Data Types:\n:', df.dtypes)
    
    if timestamp_col is not None:
        print('\nDate Range: \n\nStart:\t', df[timestamp_col].min())
        print('End:\t', df[timestamp_col].max())
        print('Days:\t',(df[timestamp_col].max() - df[timestamp_col].min()))

In [ ]:
overview(df, timestamp_col='time')

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["CFWInletPress"], mode='lines', name='CFWInletPress'))

fig.add_trace(go.Scatter(x=df["time"], y=df["CFWInletTemp"], mode='lines', name='CFWInletTemp', yaxis='y2'))

fig.update_layout(title_text="CFWInletPress vs CFWInletTemp",
                  yaxis1=dict(title="CFWInletPress", side='left'),
                  yaxis2=dict(title="CFWInletTemp", side='right', anchor="x", overlaying="y")
                  )

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["CFWInletPress"], mode='lines', name='CFWInletPress'))


fig.add_trace(go.Scatter(x=df["time"], y=df["CFWOutletTempA"], mode='lines', name='CFWOutletTempA', yaxis='y2'))

fig.update_layout(title_text="CFWInletPress vs CFWOutletTempB",
                  yaxis1=dict(title="CFWInletPress", side='left'),
                  yaxis2=dict(title="CFWOutletTempB", side='right', anchor="x", overlaying="y")
                  )

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["CFWOutletTempA"], mode='lines', name='CFWOutletTempA'))
fig.add_trace(go.Scatter(x=df["time"], y=df["CFWOutletTempB"], mode='lines', name='CFWOutletTempB', yaxis='y2'))
fig.update_layout(title_text="CFWOutletTempA vs CFWOutletTempB",
                  yaxis1=dict(title="CFWOutletTempA", side='left'),
                  yaxis2=dict(title="CFWOutletTempB", side='right', anchor="x", overlaying="y")
                  )

# Cylinder Analysis [1 - 9]

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["Cy1ExhGasOutletTemp"], mode='lines', name='Cylinder 1'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy2ExhGasOutletTemp"], mode='lines', name='Cylinder 2', yaxis='y2'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy3ExhGasOutletTemp"], mode='lines', name='Cylinder 3', yaxis='y2'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy4ExhGasOutletTemp"], mode='lines', name='Cylinder 4', yaxis='y2'))
fig.update_layout(title_text="Cylinders 1 vs 2 vs 3 vs 4",
                  yaxis1=dict(title="Cylinder Values", side='left'),
                  yaxis2=dict(title="", side='right', anchor="x", overlaying="y")
                  )

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["Cy5ExhGasOutletTemp"], mode='lines', name='Cylinder 5'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy6ExhGasOutletTemp"], mode='lines', name='Cylinder 6', yaxis='y2'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy7ExhGasOutletTemp"], mode='lines', name='Cylinder 7', yaxis='y2'))
fig.add_trace(go.Scatter(x=df["time"], y=df["Cy8ExhGasOutletTemp"], mode='lines', name='Cylinder 8', yaxis='y2'))
fig.update_layout(title_text="Cylinders 5 vs 6 vs 7 vs 8",
                  yaxis1=dict(title="Cylinder Values", side='left'),
                  yaxis2=dict(title="", side='right', anchor="x", overlaying="y")
                  )

# FOInletTemp	InletAirTemp	LOInletTemp

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["FOInletTemp"], mode='lines', name='FOInletTemp'))
fig.add_trace(go.Scatter(x=df["time"], y=df["InletAirTemp"], mode='lines', name='InletAirTemp', yaxis='y2'))
fig.add_trace(go.Scatter(x=df["time"], y=df["LOInletTemp"], mode='lines', name='LOInletTemp', yaxis='y2'))

fig.update_layout(title_text="FOInletTemp vs InletAirTemp vs LOInletTemp",
                  yaxis1=dict(title=" Values", side='left'),
                  yaxis2=dict(title="", side='right', anchor="x", overlaying="y")
                  )

# ScavPress vs	ScavTemp

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["time"], y=df["ScavPress"], mode='lines', name='ScavPress'))
fig.add_trace(go.Scatter(x=df["time"], y=df["ScavTemp"], mode='lines', name='ScavTemp', yaxis='y2'))

fig.update_layout(title_text="ScavPress vs ScavTemp",
                  yaxis1=dict(title=" Values", side='left'),
                  yaxis2=dict(title="", side='right', anchor="x", overlaying="y")
                  )